In [2]:
%matplotlib widget
import numpy as np
import rainbow.math.vector3 as V3
import rainbow.geometry.volume_mesh as VM
import rainbow.simulators.prox_soft_bodies.api as API
import rainbow.simulators.prox_soft_bodies.solver as SOLVER
import rainbow.util.viewer as VIEWER
from rainbow.util.USD import USD

In [4]:
# This block includes the helper functions to create the soft beam and the wall, and to initialize the viewer, and to simulate the scene.

# colors for visualization
COLOR_MAP = {
    'beam': V3.make(0.1, 0.8, 0.1),
    'wall': V3.make(224/255, 208/255, 193/255),
    'ground': V3.make(224/255, 208/255, 193/255)
}

# create a soft material into a scene
def create_soft_material(engine, model=API.SNH):
    # E = 10e5  # Young modulus
    # nu = 0.3  # Poisson ratio
    # rho = 1000  # Mass density
    E, nu, rho = API.create_material_parameters() # default values
    API.create_material(engine, 'soft_mat1')
    API.set_elasticity(engine, 'soft_mat1', E, nu)
    API.set_mass_density(engine, 'soft_mat1', rho)
    API.set_constitutive_model(engine, 'soft_mat1', API.SNH)
    API.set_viscosity(engine, 'soft_mat1', 0.5)
    API.create_surfaces_interaction(engine,'soft_mat1','soft_mat1', 0.5)

# create the soft beam into a scene
def create_soft_beam(engine, gravity=-10, material='soft_mat1'):
    V_beam, T_beam = VM.create_beam(20, 3, 3, 4.0, 1.0, 1.0) # geometry 
    API.create_soft_body(engine, 'beam', V_beam, T_beam)
    API.set_type(engine, 'beam', "Free")
    API.set_gravity(engine, 'beam', (0,gravity,0))
    API.set_material(engine,'beam', material)

# create the wall into a scene
def create_wall(engine):
    V_wall, T_wall = VM.create_beam(2, 2, 2, 0.1, 8.0, 8.0)
    V_wall[:,0] -= 2.05
    API.create_soft_body(engine, 'wall', V_wall, T_wall)
    API.set_type(engine, 'wall', "Fixed")
    API.set_gravity(engine, 'wall', (0,0,0))
    API.set_material(engine,'wall','soft_mat1')

# create the ground into a scene
def create_ground(engine):
    V_wall, T_wall = VM.create_beam(2, 2, 2, 8.0, 0.1, 8.0)
    V_wall[:,1] -= 0.55
    API.create_soft_body(engine, 'ground', V_wall, T_wall)
    API.set_type(engine, 'ground', "Fixed")
    API.set_gravity(engine, 'ground', (0,0,0))
    API.set_material(engine,'ground','soft_mat1')

# initialize the viewer
def initialize_viewer(engine, viewer, usd):
    # viewer.create_frame("origin")
    for body in engine.bodies.values():    
        opacity = 0.5 if body.name == "beam" else 1.0
        F = VM.to_triangles(body.T)
        color = COLOR_MAP[body.name]
        viewer.create_mesh(body.name, body.x0, F, color, opacity)   
        viewer.create_scatter(body.name + '.DBC')
        viewer.create_quiver(body.name + '.Ft')
        # viewer.hide_quiver(body.name + '.Ft')
        # add mesh to usd
        usd.add_mesh(body.name, body.x0, F)

    viewer.show()

# simulate the scene
def simulate(engine, viewer, usd, T = 1.0, update_traction=False):
    dt = engine.params.time_step
    fps = 1.0/dt
    steps = int(np.round(T*fps)) 
    usd.set_animation_time(T*fps)
    for i in range(steps):
        print("progress:{}/{}".format(i,steps), end="\r")
        API.simulate(engine, dt, debug_on=True)
        for body in engine.bodies.values():
            if body.name == "wall":
                continue
            viewer.update_mesh(body.name, body.x)
            if body.Ft is not None:
                viewer.update_quiver(body.name + '.Ft', V=body.x, N=body.Ft, scale=0.1, color=[0.7, 0.7, 0.1])
            usd.update_mesh_positions(body.name, body.x, i*dt*fps)
            if len(body.dirichlet_conditions)>0:
                indices = [bc.idx for bc in body.dirichlet_conditions]
                V = body.x[indices]
                viewer.update_scatter(body.name + '.DBC', V=V, scale=0.5, color=[0.4, 0.4, 0.4])
    return API.get_log(engine)


## Scene1
A soft beam comes into contact with the wall, and assign it a gravitational force of -10, then simulate the process of the soft beam gradually fall down.






In [3]:
# scene1 setup
scene1 = API.create_engine()
scene1.params.time_step = 0.001
viewer1 = VIEWER.Viewer()

create_soft_material(scene1) 
create_soft_beam(scene1)
create_wall(scene1)

def left_wall(x):
    return x[0] + 1.9

API.create_dirichlet_conditions(scene1, 'beam', left_wall)



usd = USD("soft_body_beam_1.usda")


initialize_viewer(scene1, viewer1, usd)
stats = simulate(scene1, viewer1, usd)

usd.save()

Renderer(camera=PerspectiveCamera(aspect=1.25, children=(DirectionalLight(color='white', intensity=0.6, positi…

/home/qbp758/anaconda3/envs/sim/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/qbp758/anaconda3/envs/sim/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
/home/qbp758/dev/RAINBOW/python/rainbow/math/vector3.py:105: RuntimeWarning: invalid value encountered in divide
  return a / np.linalg.norm(a)


# Scene2
A soft beam rests on the ground, its left side touching and fixed to the left wall. Next, we simulate a traction force pulling the soft beam horizontally from left to right.

In [5]:
def create_soft_material_2(engine, model=API.SNH):
    E, nu, rho = API.create_material_parameters("copper") 
    API.create_material(engine, 'soft_mat2')
    API.set_elasticity(engine, 'soft_mat2', E, nu)
    API.set_mass_density(engine, 'soft_mat2', rho)
    API.set_constitutive_model(engine, 'soft_mat2', model)
    API.set_viscosity(engine, 'soft_mat2', 0.5)
    API.create_surfaces_interaction(engine,'soft_mat2','soft_mat2', 0.5)


def create_soft_flat(engine, gravity=-10, material='soft_mat1'):
    V_beam, T_beam = VM.create_beam(20, 2, 2, 4.0, 0.1, 0.1) # geometry 
    API.create_soft_body(engine, 'beam', V_beam, T_beam)
    API.set_type(engine, 'beam', "Free")
    API.set_gravity(engine, 'beam', (0,gravity,0))
    API.set_material(engine,'beam', material)

# scene2 setup
scene2 = API.create_engine()
scene2.params.time_step = 0.001
viewer2 = VIEWER.Viewer()

create_soft_material(scene2, API.SVK)
create_soft_flat(scene2, 0, "soft_mat1")

def left_beam(x):
    return x[0] + 1.9

def right_beam(x):
    return 1.9 - x[0]

def center_beam(x):
    return -1 if x[0] < 0.05 or x[0] > -0.05 else 1


# set the dirichlet conditions for the beam
# API.create_dirichlet_conditions(scene2, 'beam', left_beam)
# API.create_dirichlet_conditions(scene2, 'beam', right_beam)

# set the traction force for the beam
# API.create_traction_conditions(scene2, 'beam', center_beam, load=V3.make(0.00001,3000,0))
API.create_traction_conditions(scene2, 'beam', left_beam, load=V3.make(2000, -2000, 0))
API.create_traction_conditions(scene2, 'beam', right_beam, load=V3.make(-2000, -2000, 0))


usd2 = USD("soft_body_beam_2.usda")
initialize_viewer(scene2, viewer2, usd2)

stats = simulate(scene2, viewer2, usd2, 1)

usd2.save()

Renderer(camera=PerspectiveCamera(aspect=1.25, children=(DirectionalLight(color='white', intensity=0.6, positi…

/home/qbp758/anaconda3/envs/sim/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/qbp758/anaconda3/envs/sim/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
/home/qbp758/dev/RAINBOW/python/rainbow/math/vector3.py:105: RuntimeWarning: invalid value encountered in divide
  return a / np.linalg.norm(a)
